In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
from collections import Counter
import ast

warnings.filterwarnings('ignore')
tqdm.pandas()

In [85]:
# internal data setting
internal = pd.DataFrame(columns = ["customer_id", "cluster", "sequence", "names", "tfidf", "tfidf_name"])
for num in tqdm(range(33)):
    temp = pd.read_csv(f"C:/Users/amore/Changhyun/vector/result{num}.csv")
    tfidf = pd.read_csv(f"C:/Users/amore/Changhyun/named_tfidf_results{num}.csv")
    
    indice = []
    vc = temp["cluster"].value_counts()
    for i in vc.index:
        if vc.loc[i] >= 1000:
            indice.append(i)
    for i in indice:
        temp2 = temp[temp["cluster"] == i][["customer_id", "cluster", "sequence", "names"]]
        temp2["cluster"] = [str(num) + "-" + str(w) for w in temp2["cluster"]]
        temp2["tfidf"] = [list(tfidf["item" + str(i)]) for w in temp2["cluster"]]
        temp2["tfidf_name"] = [list(tfidf["name" + str(i)]) for w in temp2["cluster"]]
        internal = pd.concat([internal, temp2], axis = 0)

100%|██████████| 33/33 [00:10<00:00,  3.11it/s]


In [89]:
# external data setting
external = pd.read_csv("result8.csv")
temp = dict(external["cluster"].value_counts())
vc = {}
n = 0
for i in temp:
    if temp[i] >= 50:
        vc[i] = n
        n += 1
    else:
        pass
num = []
for i in external["cluster"]:
    try:
        num.append(vc[i])
    except:
        num.append(np.nan)
external["cluster_num"] = num
external = external.dropna().reset_index(drop = True)
external["cluster"] = external["cluster"].apply(lambda x : ast.literal_eval(x))
external["cluster_num"] = external["cluster_num"].apply(lambda x : int(x))

In [91]:
# 추가 external 처리
from itertools import combinations
from collections import Counter

def find_most_common_subset(sets):
    # 원소의 개수가 4개 이상인 set만 필터링합니다.
    filtered_sets = [s for s in sets if len(s) >= 4]
    
    # 3개 부분집합들을 저장하기 위한 Counter를 정의합니다.
    subset_counter = Counter()
    
    # 모든 부분집합을 생성하여 카운팅합니다.
    for s in filtered_sets:
        for subset in combinations(s, 3):
            subset_counter[subset] += 1
    
    # 가장 많이 카운팅 된 3개 부분집합을 찾습니다.
    if subset_counter:
        most_common_subset, count = subset_counter.most_common(1)[0]
        return set(most_common_subset)
    else:
        return None

external2 = pd.DataFrame(columns = ["Unnamed: 0", "cluster", "cluster_num"])
idx = 0
n = 0
p = 1
while len(external.index) != 0:
    temp = find_most_common_subset(external["cluster"])
    left = []
    for a, i in enumerate(external["cluster"]):
        if temp.issubset(set(i)):
            left.append(0)
            external2.loc[idx] = [external["Unnamed: 0"].loc[a], tuple(temp), n]
            idx += 1
            if idx%1000 == 0:
                print(f"{p * 1000}번째 인덱스 달성!")
                p += 1
            else:
                pass
        else:
            left.append(1)
    external["left"] = left
    external = external[external["left"] == 1].reset_index(drop = True)
    del external["left"]
    n += 1
print(external2["cluster_num"].value_counts())
external = external2.copy()

1000번째 인덱스 달성!
2000번째 인덱스 달성!
3000번째 인덱스 달성!
4000번째 인덱스 달성!
5000번째 인덱스 달성!
6000번째 인덱스 달성!
7000번째 인덱스 달성!
8000번째 인덱스 달성!
9000번째 인덱스 달성!
10000번째 인덱스 달성!
11000번째 인덱스 달성!
cluster_num
0    6134
1    2179
2     903
3     729
4     677
5     669
6     165
7     107
Name: count, dtype: int64


In [93]:
# merge
med = pd.read_csv("mediator.csv")
med = dict(zip(med["INCS_NO"], med["SEG_KEY"]))

seg_key = []
external_cluster = []
external_cluster_num = []
for i in tqdm(internal["customer_id"]):
    seg_key.append(med[i])
    try:
        external_cluster.append(external[external["Unnamed: 0"] == med[i]]["cluster"].iloc[0])
    except:
        external_cluster.append(np.nan)
    
    try:
        external_cluster_num.append(external[external["Unnamed: 0"] == med[i]]["cluster_num"].iloc[0])
    except:
        external_cluster_num.append(np.nan)

internal["seg_key"] = seg_key
internal["external_cluster"] = external_cluster
internal["external_cluster_num"] = external_cluster_num

internal = internal.dropna().reset_index(drop = True)

100%|██████████| 355448/355448 [02:49<00:00, 2093.84it/s]


In [94]:
thres = internal.groupby(["cluster", "external_cluster_num"])["customer_id"].count().sort_values(ascending = False)
idx = []
for i in thres.index:
    if thres.loc[i] >= 32:
        idx.append(i)

final = pd.DataFrame(columns = internal.columns)
for a, b in tqdm(idx):
    temp = internal[(internal["cluster"] == a) & (internal["external_cluster_num"] == b)]
    final = pd.concat([final, temp], axis = 0)
final = final.reset_index(drop = True)
final.to_csv("merged_data.csv") # 내부 x 외부

100%|██████████| 97/97 [00:00<00:00, 189.22it/s]


,customer_id,cluster,sequence,names,tfidf,tfidf_name,seg_key,external_cluster,external_cluster_num
0,699cb2f62f368a4bcb8faf4cec63d6592f6ef5e48cbd60...,6-0,"['111772235', '111774223', '111770411', '11150...","['이니스프리 노세범미네랄팩트', '이니스프리 애플씨드클렌징크림', '이니스프리 수...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제...",6211278,"(PER_MDLV 상, PER_OTT 상, PER_PRC_SNSV 상)",0.0
1,d5ecc2bf8426946958f568e86f6f17adbec6540a308959...,6-0,"['131173630', '111791677', '111770035', '11177...","['이니스프리 트루히알루론수분자차선크림', '111791677', '이니스프리 퍼펙...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제...",6320663,"(PER_MDLV 상, PER_OTT 상, PER_PRC_SNSV 상)",0.0
2,662f112160158cbb9da36887f24a2581696ac78085a82f...,6-0,"['111500549', '111705219', '111705312', '13117...","['이니스프리 올리브리얼로션', '이니스프리 올리브리얼스킨', '이니스프리 그린티밸...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제...",6330067,"(PER_MDLV 상, PER_OTT 상, PER_PRC_SNSV 상)",0.0
3,2a9a580f05e866ae2f4127342ee6905590460e0c7048db...,6-0,"['111506419', '131174321', '111700874', '13117...","['111506419', '이니스프리 그린티히알루론산스킨', '이니스프리 그린티클렌...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제...",6220407,"(PER_MDLV 상, PER_OTT 상, PER_PRC_SNSV 상)",0.0
4,e314e8d74ebbd0458995b3536320d9794354831db4c0b0...,6-0,"['111770033', '111770034', '111772219', '11177...","['이니스프리 퍼펙트9리페어세럼', '이니스프리 퍼펙트9리페어크림', '이니스프리 ...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제...",6310637,"(PER_MDLV 상, PER_OTT 상, PER_PRC_SNSV 상)",0.0
...,...,...,...,...,...,...,...,...,...
11532,f095e9c4711dab0274daf02673ac46687fa3379414bcb6...,1-0,['111774223'],['이니스프리 애플씨드클렌징크림'],"[111770034, 111705219, 131172564, 131173332, 1...","[이니스프리 퍼펙트9리페어크림, 이니스프리 올리브리얼스킨, 이니스프리 블랙티유스인핸...",1211465,"(LFSTG 05.성인자녀추정, JOB 02_회사원, PER_MDLV 상)",5.0
11533,1d18c7072b56816852b3e40da62c42a8eb9bb69da253e5...,1-0,['111771793'],['이니스프리 리얼컬러네일'],"[111770034, 111705219, 131172564, 131173332, 1...","[이니스프리 퍼펙트9리페어크림, 이니스프리 올리브리얼스킨, 이니스프리 블랙티유스인핸...",1211248,"(LFSTG 05.성인자녀추정, JOB 02_회사원, PER_MDLV 상)",5.0
11534,3a2f98527824f0e2d5b83d980351f1225180fb2871987d...,1-0,"['111705377', '111771038', '111772219']","['이니스프리 화산송이모공폼', '이니스프리 꽃송이버섯바이탈로션', '이니스프리 에...","[111770034, 111705219, 131172564, 131173332, 1...","[이니스프리 퍼펙트9리페어크림, 이니스프리 올리브리얼스킨, 이니스프리 블랙티유스인핸...",1220492,"(LFSTG 05.성인자녀추정, JOB 02_회사원, PER_MDLV 상)",5.0
11535,1c48584732a5775a226b8d687433f6136cbf1d7d6a8779...,1-0,"['131175032', '111770997', '131173960']","['이니스프리 레티놀그린티피디알엔앰플', '이니스프리 잇츠리얼스퀴즈마스크', '이니...","[111770034, 111705219, 131172564, 131173332, 1...","[이니스프리 퍼펙트9리페어크림, 이니스프리 올리브리얼스킨, 이니스프리 블랙티유스인핸...",1120803,"(LFSTG 05.성인자녀추정, JOB 02_회사원, PER_MDLV 상)",5.0


In [29]:
# 내부 추출
final = pd.read_csv("merged_data.csv", index_col = 0)
in_ex = list(final["customer_id"])

internal["filter"] = [1 if i in in_ex else 0 for i in tqdm(internal["customer_id"])]
internal = internal[internal["filter"] == 0].reset_index(drop = True)

in_ex = list(final["cluster"].unique())
internal["filter"] = [1 if i in in_ex else 0 for i in tqdm(internal["cluster"])]
internal = internal[internal["filter"] == 1].reset_index(drop = True)

100%|██████████| 343911/343911 [00:00<00:00, 1987978.53it/s]


In [57]:
vc = final["cluster"].value_counts()
internal_only = pd.DataFrame(columns = internal.columns)
for i in tqdm(in_ex):
    temp = internal[internal["cluster"] == i].sample(frac = 1).reset_index(drop = True).loc[:vc[i]]
    internal_only = pd.concat([internal_only, temp], axis = 0)
internal_only = internal_only.reset_index(drop = True)
internal_only

100%|██████████| 32/32 [00:01<00:00, 26.89it/s]


,customer_id,cluster,sequence,names,tfidf,tfidf_name,filter
0,82e25362916e3c95ffb476afb2f950fc06cf9091fe9daf...,6-0,"['111770997', '131172563', '111776682', '13117...","['IF. 잇츠리얼스퀴즈마스크-오이 20ML', 'IF. 블랙티 유스 인핸싱 앰플 ...","[131172880, 131174283, 131172879, 131171020, 1...","[IF.리스테이 카밍 컨디셔너 480ml(R), IF. 아일넘버 핸드크림 씨즈더모먼...",1
1,f9ec5a8eb803d2ad9def33380246369ef2c16280e3210d...,6-0,"['131171726', '111770183', '131173960', '13117...","['IF.포레스트 포맨 2종 세트 (19)', '이니스프리 유채꿀립밤', 'IF. ...","[131172880, 131174283, 131172879, 131171020, 1...","[IF.리스테이 카밍 컨디셔너 480ml(R), IF. 아일넘버 핸드크림 씨즈더모먼...",1
2,da4f13f18fcab91144d48a66d479c821eb365ad592cb56...,6-0,"['131173569', '111506404', '131173569', '13117...","['IF. 레티놀 시카 흔적 앰플 30ml', '이니스프리 에코뷰티툴일반화장솜', ...","[131172880, 131174283, 131172879, 131171020, 1...","[IF.리스테이 카밍 컨디셔너 480ml(R), IF. 아일넘버 핸드크림 씨즈더모먼...",1
3,24d073d27d8c2b8dba60960ee9037cc0525bb1ce6c4aef...,6-0,"['131171316', '111705279', '111770419', '11177...","['IF.포어 블러 파우더 11g', '이니스프리 더그린티씨드세럼', '이니스프리 ...","[131172880, 131174283, 131172879, 131171020, 1...","[IF.리스테이 카밍 컨디셔너 480ml(R), IF. 아일넘버 핸드크림 씨즈더모먼...",1
4,9da79ec2ffcede0258a71cbfb1ffb6ae23ab7592deea9e...,6-0,"['111770997', '131173707', '111776178', '11177...","['IF. 잇츠리얼스퀴즈마스크-오이 20ML', 'IF. 모이스처 실크 베이스 1 ...","[131172880, 131174283, 131172879, 131171020, 1...","[IF.리스테이 카밍 컨디셔너 480ml(R), IF. 아일넘버 핸드크림 씨즈더모먼...",1
...,...,...,...,...,...,...,...
11564,37b931fb2471ae177a239abc57d6a5cdb82602b712d6f8...,4-0,"['131173534', '111770183']","['IF. (글로벌)듀이 틴트 립밤 1 3.2G', '이니스프리 유채꿀립밤']","[131173960, 111771793, 131173534, 111770183, 1...","[IF. 비타C그린티엔자임잡티토닝세럼 30ML (23), 이니스프리 리얼컬러네일, ...",1
11565,88732a4f8c22a75c61caffe9243f4e5b7ad3618aecc10f...,4-0,['131170000'],['IF. 스키니 꼼꼼카라 Zero 1 블랙 3.5G'],"[131173960, 111771793, 131173534, 111770183, 1...","[IF. 비타C그린티엔자임잡티토닝세럼 30ML (23), 이니스프리 리얼컬러네일, ...",1
11566,76f6f54dfa91e58b54a48594629e3d5a5cdedd00bd1f50...,4-0,['131173534'],['IF. (글로벌)듀이 틴트 립밤 1 3.2G'],"[131173960, 111771793, 131173534, 111770183, 1...","[IF. 비타C그린티엔자임잡티토닝세럼 30ML (23), 이니스프리 리얼컬러네일, ...",1
11567,8fe8439161c2755a18e8ff525a8370fc5413c31943ff02...,4-0,['111771154'],['이니스프리 알로에리바이탈수딩젤'],"[131173960, 111771793, 131173534, 111770183, 1...","[IF. 비타C그린티엔자임잡티토닝세럼 30ML (23), 이니스프리 리얼컬러네일, ...",1


In [67]:
internal_only.to_csv("internal_only.csv", encoding = "utf-8-sig")